# Imports & Definitions

In [1]:
from SVD_utils import *
from Error_utils import *
import json
import numpy as np

# Creating arrayD

In [3]:
deleteArrays(D=True)
createDB("arrayD", "Temperatures", (num_images, image_height, image_width), (tile_thickness, tile_height, tile_width))
createDB("arrayD_prime", "Temperatures", (num_images, image_height, image_width), (tile_thickness, tile_height, tile_width))
max_val, min_val, data_avg = populateArrayD()
vRange = max_val - min_val

# SVD related work

In [3]:
deleteArrays(D=False)
createDB("arrayUr", "U", db_shape=(num_SVD_matrices, SVD_shape[0], k), tile_shape=(1,SVD_shape[0],k), data_type=np.float32)
createDB("arraySr", "S", db_shape=(num_SVD_matrices, k, 1), tile_shape=(1,k,1), data_type=np.float32)
createDB("arrayVr", "V", db_shape=(num_SVD_matrices, k, SVD_shape[1]), tile_shape=(1,k,SVD_shape[1]), data_type=np.float32)

In [7]:
max_error, min_error, percent_error = compressData(data_avg, vRange)

100%|██████████| 855/855 [03:42<00:00,  3.85it/s]


In [8]:
# check quantizability
do_quant = 0

for i in [2**8, 2**16]:
    step = (max_error - min_error) / i
    
    if (step/2) / vRange < eps:
        do_quant = i 
        break
    
print(do_quant)

E_datatype = np.float32

if do_quant == 2**8:
    E_datatype = np.uint8
elif do_quant == 2**16:
    E_datatype = np.uint16

256


In [9]:
step = (max_error - min_error) / 256
print((step/2) / vRange)

0.0005746626849361167


In [10]:
print(percent_error)

#decide whether to be sparse here:
is_sparse = True if percent_error < 0.33 else False

0.06498722450845908


In [11]:
max_error - min_error

24.997692108154297

In [12]:
createDB("arrayE", "E", (num_SVD_matrices, SVD_shape[0], SVD_shape[1]), (1, SVD_shape[0], SVD_shape[1]), sparse=is_sparse, data_type=E_datatype)
populateErrorArray(max_error, min_error, vRange, data_avg, do_quant, E_datatype, is_sparse)

100%|██████████| 855/855 [05:02<00:00,  2.83it/s]


In [35]:
reconstructAndCheck(max_error, min_error, vRange, data_avg, do_quant, is_sparse)

100%|██████████| 1215/1215 [02:56<00:00,  6.87it/s]

Check successful


# JSON and Compression

In [13]:
getCompressionRatio()

16624538526
726271427 684072194
Compression ratio p: 11.787580188587246


11.787580188587246

In [5]:
param_dict = {}
parameter_names = ["max_val", "min_val", "data_avg", "vRange", "do_quant", "max_error", "min_error", "E_datatype"]

for i in parameter_names:
    if i == "E_datatype":
        param_dict[i] = str(globals()[i])

    else: param_dict[i] = float(globals()[i])

with open("parameters.json", "w") as f:
    json.dump(param_dict, f)

KeyError: 'max_val'

In [6]:
with open("parameters.json", "r") as f:
    param_dict = json.load(f)

for key, value in param_dict.items():
    if key == "E_datatype":
        if "8" in value:
            E_datatype = np.uint8
        elif "16" in value:
            E_datatype = np.uint16
        else: E_datatype = np.float32
    else:
        globals()[key] = np.float32(value)

# Extras

In [5]:
param_dict = {}
parameter_names = ["max_val", "min_val", "data_avg", "vRange"]

for i in parameter_names:
    param_dict[i] = float(globals()[i])

with open("parameters.json", "w") as f:
    json.dump(param_dict, f)

In [2]:
with open("parameters.json", "r") as f:
    param_dict = json.load(f)

for key, value in param_dict.items():
    globals()[key] = np.float32(value)